In [ ]:
!conda info --envs

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import scanpy as sc

In [ ]:
import scvi

In [ ]:
import seaborn as sns

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
out = []

In [ ]:
data = !ls preprocessed_h5ad_data_integrated/
data_ad = ["preprocessed_h5ad_data_integrated/" + x for x in data]

In [ ]:
len(data)

In [ ]:
#import preprocessed files
for file in data_ad:
    adata = sc.read_h5ad(file)
    adata.var_names_make_unique()
    out.append(adata)

In [ ]:
out

In [ ]:
adata = sc.concat(out, join="outer")

In [ ]:
adata

In [ ]:
adata.obs

In [ ]:
obs = adata.obs
obs.index = obs["sample_id"]

In [ ]:
obs

In [ ]:
clinical = pd.read_csv("Data_Summary_30.csv")

In [ ]:
clinical

In [ ]:
clinical.index = clinical["adata.obs."]

In [ ]:
clinical

In [ ]:
concat = pd.concat([obs, clinical], axis=1).reindex(obs.index)

In [ ]:
concat

In [ ]:
concat.index = concat["cell_id"]
concat.index.name = None
concat

In [ ]:
adata.obs = concat

In [ ]:
adata

In [ ]:
sc.pp.filter_genes(adata, min_cells = 100)

In [ ]:
adata

In [ ]:
adata.X

In [ ]:
adata.layers['counts'] = adata.X.copy()

In [ ]:
sc.pp.normalize_total(adata, target_sum = 1e4)
sc.pp.log1p(adata)
adata.raw = adata

In [ ]:
adata

In [ ]:
adata.obs.head()

In [ ]:
adata.obs.columns

In [ ]:
scvi.model.SCVI.setup_anndata(adata, layer = "counts",
                             categorical_covariate_keys=["sample_id", "GSE_ID"],
                             continuous_covariate_keys=["pct_counts_mt", "total_counts"])

In [ ]:
model = scvi.model.SCVI(adata)

In [ ]:
model.train()

In [ ]:
model.get_latent_representation()

In [ ]:
model.get_latent_representation().shape

In [ ]:
adata.obsm["X_scVI"] = model.get_latent_representation()

In [ ]:
model.get_normalized_expression(library_size = 1e4)

In [ ]:
adata.layers['scvi_normalized'] = model.get_normalized_expression(library_size = 1e4)

In [ ]:
adata

In [ ]:
# save the data
adata.write_h5ad("hgsoc_integrated_scvi_sampleID_GSEID.h5ad")

In [ ]:
!ls

In [ ]:
!pip install --quiet scvi-colab
from scvi_colab import install

install()

In [ ]:
import gdown
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import scvi
import seaborn as sns
from scvi.external import CellAssign

In [ ]:
marker_gene_mat_hgsc = pd.read_csv("HGSC_celltype.csv", index_col=0)

In [ ]:
lib_size = adata.X.sum(1)
adata.obs["size_factor"] = lib_size / np.mean(lib_size)

In [ ]:
bdata = adata[:, marker_gene_mat_hgsc.index].copy()

In [ ]:
scvi.external.CellAssign.setup_anndata(bdata, "size_factor")

In [ ]:
model_hgsc = CellAssign(bdata, marker_gene_mat_hgsc)
model_hgsc.train()

In [ ]:
model_hgsc.history["elbo_validation"].plot()

In [ ]:
predictions_hgsc = model_hgsc.predict()

In [ ]:
sns.clustermap(predictions_hgsc, cmap="viridis")

In [ ]:
bdata.obs["scvi-tools predictions"] = predictions_hgsc.idxmax(axis=1).values

In [ ]:
sc.pl.umap(
    bdata,
    color=["scvi-tools predictions"],
    ncols=1,
    frameon=False,
)